In [208]:
%matplotlib inline

import sys
import time
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.covariance import GraphicalLasso as sk_GL
from sklearn.covariance import empirical_covariance

sys.path.append('..')
from gglasso.solver.single_admm_solver import ADMM_SGL
from gglasso.solver.single_admm_solver import block_SGL
from helper.data_generation import time_varying_power_network, group_power_network, sample_covariance_matrix
from helper.model_selection import single_grid_search


from regain.covariance import GraphicalLasso as rg_GL

# Sampling from power networks

In [209]:
p = 1000
N = 2000
Sigma, Theta = group_power_network(p, K=5, M=2) #Theta is true precision matrix

S, samples = sample_covariance_matrix(Sigma, N)

S = S[0,:,:]
Theta = Theta[0,:,:] #true precision matrix

samples.shape

(5, 1000, 2000)

In [210]:
lambda1 = 0.01
Omega_0 = np.eye(p)

n_iter = 50000
tol_list = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]

## Model solution Z

In [211]:
start = time.time()
model = rg_GL(alpha=lambda1, max_iter=n_iter, tol = 1e-10).fit(samples[0,:,:].T)
end = time.time()

hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
Z_time = "{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)
print(Z_time)

00:00:16.62


In [212]:
Z = model.precision_

In [213]:
start = time.time()
sol, info = ADMM_SGL(S, lambda1, Omega_0, max_iter=n_iter, 
                     tol = 1e-10, rtol = 1e-10, stopping_criterion = "boyd",
                     verbose = False, latent = False)
end = time.time()

hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
Z_time = "{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)
print(Z_time)

ADMM terminated after 58 iterations with accuracy 4.982706427489555e-05
ADMM status: {'dual optimal', 'primal optimal'}
00:00:17.41


In [214]:
Z1 = sol["Theta"]

In [215]:
np.linalg.norm(Z1-Z)

0.005684772361137492

In [216]:
# [1e-1, 5e-2, 25e-3, 1e-2,5e-3, 25e-4, 1e-3, 5e-4]

## Sklearn

In [217]:
cov_list = []
precision_list = []
time_list = []
for tol in tol_list:
    start = time.time()
    Z_n = sk_GL(alpha=lambda1, max_iter=n_iter, tol = tol, enet_tol = 0.1).fit(samples[0,:,:].T)
    end = time.time()
    
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    time_list.append("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    
    cov_list.append(Z_n.covariance_)
    precision_list.append(Z_n.precision_)
    
accuracy_list = []
for i in range(0, len(precision_list)):
    accuracy = np.linalg.norm(Z - np.array(precision_list[i]))/np.linalg.norm(Z)
    accuracy_list.append(accuracy)

In [218]:
str_tol = [str(x) for x in tol_list]
method_list = ["sklearn"]*len(accuracy_list)

df_sk = pd.DataFrame(data={'time': time_list, 
                           'distance': accuracy_list,
                           'method': method_list,
                           'tol_rate': tol_list,
                            'str_tol': str_tol})
df_sk

,time,distance,method,tol_rate,str_tol
0,00:00:08.96,0.002712,sklearn,1.000000e-01,0.1
1,00:00:25.51,0.000355,sklearn,1.000000e-02,0.01
2,00:00:36.64,0.000136,sklearn,1.000000e-03,0.001
3,00:00:44.69,0.000126,sklearn,1.000000e-04,0.0001
4,00:00:53.01,0.000125,sklearn,1.000000e-05,1e-05
5,00:00:50.46,0.000125,sklearn,1.000000e-06,1e-06
6,00:01:27.88,0.000125,sklearn,1.000000e-07,1e-07
7,00:02:16.31,0.000125,sklearn,1.000000e-08,1e-08


In [219]:
fig = px.line(df_sk, x="str_tol", y="distance", color= "method", log_y = True,
              labels={
                     "str_tol": "Tolerance rate",
                     "distance": "Log_distance",
                     "method": "method"
                 },
              template = "plotly_white",
              title="Log-distance between Z and Z' with respect to ADMM convergence rates")
fig.show()

## Regain

In [220]:
cov_list = []
precision_list = []
time_list = []
for tol in tol_list:
    start = time.time()
    Z_n = rg_GL(alpha=lambda1, max_iter=n_iter, tol = tol, rtol = tol).fit(samples[0,:,:].T)
    end = time.time()
    
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    time_list.append("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    
    cov_list.append(Z_n.covariance_)
    precision_list.append(Z_n.precision_)

accuracy_list = []
for i in range(0, len(precision_list)):
    accuracy = np.linalg.norm(Z - np.array(precision_list[i]))/np.linalg.norm(Z)
    accuracy_list.append(accuracy)

In [221]:
str_tol = [str(x) for x in tol_list]
method_list = ["regain"]*len(accuracy_list)

df_rg = pd.DataFrame(data={'time': time_list, 
                           'distance': accuracy_list,
                           'method': method_list,
                           'tol_rate': tol_list,
                            'str_tol': str_tol})
df_rg

,time,distance,method,tol_rate,str_tol
0,00:00:00.67,0.784532,regain,1.000000e-01,0.1
1,00:00:00.96,0.662685,regain,1.000000e-02,0.01
2,00:00:04.52,0.078428,regain,1.000000e-03,0.001
3,00:00:08.51,0.007218,regain,1.000000e-04,0.0001
4,00:00:12.66,0.000995,regain,1.000000e-05,1e-05
5,00:00:17.34,0.000019,regain,1.000000e-06,1e-06
6,00:00:20.23,0.000110,regain,1.000000e-07,1e-07
7,00:00:24.52,0.000124,regain,1.000000e-08,1e-08


## ADMM 

We run the algorithm 2 times because on the first iteration time costly numba initialization happens.

In [222]:
for i in [0,1]:
    cov_list = []
    precision_list = []
    time_list = []
    for tol in tol_list:
        start = time.time()
        sol, info = ADMM_SGL(S, lambda1, Omega_0, max_iter=n_iter, tol = tol, rtol = tol, stopping_criterion = "boyd",
                             verbose = False, latent = False)
        end = time.time()

        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        time_list.append("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

        cov_list.append(sol['Omega'])
        precision_list.append(sol['Theta'])

    accuracy_list = []
    for i in range(0, len(precision_list)):
        accuracy = np.linalg.norm(Z - np.array(precision_list[i]))/np.linalg.norm(Z)
        accuracy_list.append(accuracy)

ADMM terminated after 1 iterations with accuracy 7.201949322409392
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 1 iterations with accuracy 7.201949322409392
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 1 iterations with accuracy 7.201949322409392
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 1 iterations with accuracy 7.201949322409392
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 3 iterations with accuracy 2.8805271712760705
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 9 iterations with accuracy 0.38510134274143787
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 17 iterations with accuracy 0.04725723902173184
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 28 iterations with accuracy 0.004228150205120534
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 1 iterations with accuracy 7.201949322409392
ADM

In [223]:
str_tol = [str(x) for x in tol_list]
method_list = ["admm"]*len(accuracy_list)

df_admm = pd.DataFrame(data={'time': time_list, 
                           'distance': accuracy_list,
                           'method': method_list,
                           'tol_rate': tol_list,
                            'str_tol': str_tol})
df_admm

,time,distance,method,tol_rate,str_tol
0,00:00:00.48,0.329017,admm,1.000000e-01,0.1
1,00:00:00.47,0.329017,admm,1.000000e-02,0.01
2,00:00:00.45,0.329017,admm,1.000000e-03,0.001
3,00:00:00.33,0.329017,admm,1.000000e-04,0.0001
4,00:00:00.92,0.159649,admm,1.000000e-05,1e-05
5,00:00:02.30,0.029772,admm,1.000000e-06,1e-06
6,00:00:04.15,0.004378,admm,1.000000e-07,1e-07
7,00:00:06.85,0.000677,admm,1.000000e-08,1e-08


## kkt_ADMM

We run the algorithm 2 times because on the first iteration time costly numba initialization happens.

In [224]:
for i in [0,1]:
    cov_list = []
    precision_list = []
    time_list = []
    for tol in tol_list:
        start = time.time()
        sol, info = ADMM_SGL(S, lambda1, Omega_0, max_iter=n_iter, tol = tol, stopping_criterion = "kkt",
                             verbose = False, latent = False)
        end = time.time()

        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        time_list.append("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

        cov_list.append(sol['Omega'])
        precision_list.append(sol['Theta'])

    accuracy_list = []
    for i in range(0, len(precision_list)):
        accuracy = np.linalg.norm(Z - np.array(precision_list[i]))/np.linalg.norm(Z)
        accuracy_list.append(accuracy)

ADMM is only {'primal and dual optimal'}
Try to change the tolerance value 0.1
ADMM terminated after 2 iterations with accuracy 0.07689794887452671
ADMM status: {'primal and dual optimal'}
ADMM is only {'primal and dual optimal'}
Try to change the tolerance value 0.01
ADMM terminated after 8 iterations with accuracy 0.00924577452541569
ADMM status: {'primal and dual optimal'}
ADMM is only {'primal and dual optimal'}
Try to change the tolerance value 0.001
ADMM terminated after 17 iterations with accuracy 0.0008594751116571462
ADMM status: {'primal and dual optimal'}
ADMM is only {'primal and dual optimal'}
Try to change the tolerance value 0.0001
ADMM terminated after 27 iterations with accuracy 9.764438701732223e-05
ADMM status: {'primal and dual optimal'}
ADMM is only {'primal and dual optimal'}
Try to change the tolerance value 1e-05
ADMM terminated after 42 iterations with accuracy 9.390545098642266e-06
ADMM status: {'primal and dual optimal'}
ADMM is only {'primal and dual optimal

In [225]:
str_tol = [str(x) for x in tol_list]
method_list = ["kkt_admm"]*len(accuracy_list)

df_kkt = pd.DataFrame(data={'time': time_list, 
                           'distance': accuracy_list,
                           'method': method_list,
                           'tol_rate': tol_list,
                            'str_tol': str_tol})
df_kkt

,time,distance,method,tol_rate,str_tol
0,00:00:01.45,0.222359,kkt_admm,1.000000e-01,0.1
1,00:00:04.48,0.038702,kkt_admm,1.000000e-02,0.01
2,00:00:09.14,0.004378,kkt_admm,1.000000e-03,0.001
3,00:00:14.99,0.000772,kkt_admm,1.000000e-04,0.0001
4,00:00:28.95,0.000198,kkt_admm,1.000000e-05,1e-05
5,00:00:21.65,0.000133,kkt_admm,1.000000e-06,1e-06
6,00:00:29.13,0.000126,kkt_admm,1.000000e-07,1e-07
7,00:00:34.62,0.000126,kkt_admm,1.000000e-08,1e-08


## Block ADMM

In [226]:
for i in [0,1]:
    cov_list = []
    precision_list = []
    time_list = []
    for tol in tol_list:
        start = time.time()
        sol = block_SGL(S, lambda1, Omega_0, max_iter=n_iter, tol = tol, rtol = tol, stopping_criterion = "boyd",
                             verbose = False)
        end = time.time()

        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        time_list.append("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

        cov_list.append(sol['Omega'])
        precision_list.append(sol['Theta'])

    accuracy_list = []
    for i in range(0, len(precision_list)):
        accuracy = np.linalg.norm(Z - np.array(precision_list[i]))/np.linalg.norm(Z)
        accuracy_list.append(accuracy)

ADMM terminated after 1 iterations with accuracy 7.201949322409392
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 1 iterations with accuracy 7.201949322409392
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 1 iterations with accuracy 7.201949322409392
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 1 iterations with accuracy 7.201949322409392
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 3 iterations with accuracy 2.8805271712760705
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 9 iterations with accuracy 0.38510134274143787
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 17 iterations with accuracy 0.04725723902173184
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 28 iterations with accuracy 0.004228150205120534
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 1 iterations with accuracy 7.201949322409392
ADM

In [227]:
str_tol = [str(x) for x in tol_list]
method_list = ["block_admm"]*len(accuracy_list)

df_block = pd.DataFrame(data={'time': time_list, 
                           'distance': accuracy_list,
                           'method': method_list,
                           'tol_rate': tol_list,
                            'str_tol': str_tol})
df_block

,time,distance,method,tol_rate,str_tol
0,00:00:00.43,0.329017,block_admm,1.000000e-01,0.1
1,00:00:00.41,0.329017,block_admm,1.000000e-02,0.01
2,00:00:00.40,0.329017,block_admm,1.000000e-03,0.001
3,00:00:00.40,0.329017,block_admm,1.000000e-04,0.0001
4,00:00:00.74,0.159649,block_admm,1.000000e-05,1e-05
5,00:00:01.80,0.029772,block_admm,1.000000e-06,1e-06
6,00:00:03.39,0.004378,block_admm,1.000000e-07,1e-07
7,00:00:05.98,0.000677,block_admm,1.000000e-08,1e-08


## Block kkt_ADMM

In [228]:
for i in [0,1]:
    cov_list = []
    precision_list = []
    time_list = []
    for tol in tol_list:
        start = time.time()
        sol = block_SGL(S, lambda1, Omega_0, max_iter=n_iter, tol = tol, stopping_criterion = "kkt",
                             verbose = False)
        end = time.time()

        hours, rem = divmod(end-start, 3600)
        minutes, seconds = divmod(rem, 60)
        time_list.append("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

        cov_list.append(sol['Omega'])
        precision_list.append(sol['Theta'])

    accuracy_list = []
    for i in range(0, len(precision_list)):
        accuracy = np.linalg.norm(Z - np.array(precision_list[i]))/np.linalg.norm(Z)
        accuracy_list.append(accuracy)

ADMM is only {'primal and dual optimal'}
Try to change the tolerance value 0.1
ADMM terminated after 2 iterations with accuracy 0.07689794887452671
ADMM status: {'primal and dual optimal'}
ADMM is only {'primal and dual optimal'}
Try to change the tolerance value 0.01
ADMM terminated after 8 iterations with accuracy 0.00924577452541569
ADMM status: {'primal and dual optimal'}
ADMM is only {'primal and dual optimal'}
Try to change the tolerance value 0.001
ADMM terminated after 17 iterations with accuracy 0.0008594751116571462
ADMM status: {'primal and dual optimal'}
ADMM is only {'primal and dual optimal'}
Try to change the tolerance value 0.0001
ADMM terminated after 27 iterations with accuracy 9.764438701732223e-05
ADMM status: {'primal and dual optimal'}
ADMM is only {'primal and dual optimal'}
Try to change the tolerance value 1e-05
ADMM terminated after 42 iterations with accuracy 9.390545098642266e-06
ADMM status: {'primal and dual optimal'}
ADMM is only {'primal and dual optimal

In [229]:
str_tol = [str(x) for x in tol_list]
method_list = ["kkt_block_admm"]*len(accuracy_list)

df_block_kkt = pd.DataFrame(data={'time': time_list, 
                           'distance': accuracy_list,
                           'method': method_list,
                           'tol_rate': tol_list,
                            'str_tol': str_tol})
df_block_kkt

,time,distance,method,tol_rate,str_tol
0,00:00:01.58,0.222359,kkt_block_admm,1.000000e-01,0.1
1,00:00:04.43,0.038702,kkt_block_admm,1.000000e-02,0.01
2,00:00:09.01,0.004378,kkt_block_admm,1.000000e-03,0.001
3,00:00:14.20,0.000772,kkt_block_admm,1.000000e-04,0.0001
4,00:00:21.77,0.000198,kkt_block_admm,1.000000e-05,1e-05
5,00:00:29.01,0.000133,kkt_block_admm,1.000000e-06,1e-06
6,00:00:39.71,0.000126,kkt_block_admm,1.000000e-07,1e-07
7,00:00:41.45,0.000126,kkt_block_admm,1.000000e-08,1e-08


## Data formating

In [230]:
frames = [df_sk, df_rg, df_admm, df_kkt, df_block, df_block_kkt]

df = pd.concat(frames)

df["str_tol"].replace({"0.0001": "1e-04"}, inplace=True)
dist = np.array(df['distance'])
df['log_distance'] = np.log(dist)

In [231]:
df.sort_values(by=['log_distance'], ascending = False)

,time,distance,method,tol_rate,str_tol,log_distance
0,00:00:00.67,0.784532,regain,1.000000e-01,0.1,-0.242668
1,00:00:00.96,0.662685,regain,1.000000e-02,0.01,-0.411456
3,00:00:00.40,0.329017,block_admm,1.000000e-04,1e-04,-1.111646
2,00:00:00.40,0.329017,block_admm,1.000000e-03,0.001,-1.111646
3,00:00:00.33,0.329017,admm,1.000000e-04,1e-04,-1.111646
2,00:00:00.45,0.329017,admm,1.000000e-03,0.001,-1.111646
1,00:00:00.47,0.329017,admm,1.000000e-02,0.01,-1.111646
0,00:00:00.48,0.329017,admm,1.000000e-01,0.1,-1.111646
0,00:00:00.43,0.329017,block_admm,1.000000e-01,0.1,-1.111646
1,00:00:00.41,0.329017,block_admm,1.000000e-02,0.01,-1.111646


## Tolerance plot

In [232]:
fig = px.line(df, x="str_tol", y="distance", color= "method", 
              log_y = True,
              labels={
                     "str_tol": "Tolerance rate",
                     "distance": "Log_distance",
                     "method": "method"
                 },
              template = "plotly_white",
              title="Log-distance between Z and Z' with respect to ADMM convergence rates")
fig.show()

In [233]:
df = df.reset_index(drop=True)

for i, row in df.iterrows():
    sec_time = df['time'][i][-5:]
    df.at[i,'sec_time'] = sec_time
df['sec_time'] = pd.to_numeric(df['sec_time'])
df.head()

,time,distance,method,tol_rate,str_tol,log_distance,sec_time
0,00:00:08.96,0.002712,sklearn,0.10000,0.1,-5.910039,8.96
1,00:00:25.51,0.000355,sklearn,0.01000,0.01,-7.942614,25.51
2,00:00:36.64,0.000136,sklearn,0.00100,0.001,-8.906128,36.64
3,00:00:44.69,0.000126,sklearn,0.00010,1e-04,-8.981124,44.69
4,00:00:53.01,0.000125,sklearn,0.00001,1e-05,-8.984194,53.01


## Time plot

In [234]:
import plotly.graph_objects as go
fig = px.line(df, x="sec_time", y="distance",color='method', title='ADMM perfomance comparison', 
              template = "plotly_white",
              labels={"sec_time": "Time, s",
                "distance": "Distance",
                "method": "method"
                 })

fig.show()

In [235]:
fig = px.line(df, x="sec_time", y="distance",color='method', title='ADMM perfomance comparison', 
              template = "plotly_white",
              labels={"sec_time": "Time, s",
                "distance": "Log_distance",
                "method": "method"
                 },
              log_y = True)

fig.show()

In [236]:
fig = px.line(df, x="sec_time", y="distance", color= "method", log_y = True, line_dash='method',
#               text = "str_tol",
        labels={"sec_time": "Time, s",
                "distance": "Log_distance",
                "method": "method"
                 },
        template = "plotly_white",
              title='ADMM perfomance comparison')
fig.update_traces(textposition='top center')
fig.show()

In [237]:
fig = px.line(df, x="sec_time", y="distance", color= "method", line_dash='method',
#               text = "str_tol",
        labels={"sec_time": "Time, s",
                "distance": "Distance",
                "method": "method"
                 },
        template = "plotly_white",
              title='ADMM perfomance comparison')
fig.update_traces(textposition='top center')
fig.show()

In [238]:
# fig_1 = px.scatter(x=a['time'], y=a['accuracy'], color= "method")
# fig_1.show()

fig_1 = px.scatter(df, x="sec_time", 
                   y='distance', color="method", 
#                    text = "sec_time", 
                   log_y = True, template = "plotly_white",
                   text = "str_tol",
                 size='sec_time',
                  title='ADMM perfomance comparison')
fig_1.update_traces(textposition='top center')
fig_1.show()